<a href="https://colab.research.google.com/github/ManteLab/ItoN/blob/main/10_learning_in_biology_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise session 10: Learning in Biology

In this exercise session, we will investigate some aspects of learning mechanisms in the brain. First, we will introduce some biological principles, then discuss mathematical models that describe them and finally you will have the opportuniy to simulate these models.

## Setup

First, we need to update the widgets since we use a never version than the Colab default.


In [1]:
!pip install -U ipywidgets

Next, we download additional files required by this tutorial.

In [ ]:
!mkdir utils_ex10
!wget -P utils_ex10/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/stdp.py
!wget -P utils_ex10/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/ltp_ltd.py
!wget -P utils_ex10/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/odor.py
!wget -P utils_ex10/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/odor_plotting.py

--2024-11-20 06:23:40--  https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/stdp.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-20 06:23:40 ERROR 404: Not Found.

--2024-11-20 06:23:41--  https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/ltp_ltd.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-20 06:23:41 ERROR 404: Not Found.

--2024-11-20 06:23:41--  https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/odor.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-20 06:23:41 ERROR 404: Not Found.

--20

Install dependencies

In [ ]:
!pip install netgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 10.6 MB/s eta 0:00:00


Import the downloaded files and enable custom widgets

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [2]:
from utils_ex10.stdp import *
from utils_ex10.ltp_ltd import *
from utils_ex10.odor_plotting import *

# 1. Short-term plasticty

Source: http://www.scholarpedia.org/article/Short-term_synaptic_plasticity

Although memory formation in the brain is believed to rely primarily on long-lasting changes in synaptic strength and connectivity, shorter-lived modulation of synaptic strength, known as **short-term plasticity (STP)**, also plays an important role.

While some artificial models implement STP, it is often neglected in others. This provides us with an opportunity to extend the content of this course to explore the mechanisms underlying STP.

You have covered quite some ground on the matter of STP already during the lecture but let usre-emphasise the concepts again.

In brief, STP is a phenomenon where synaptic efficacy changes dynamically over time, based on the history of presynaptic activity. As the name suggests, these processes are relatively short-lived, typically lasting for milliseconds.

Before delving into the details of a mathematical model describing STP, we first examine its two primary forms:

**Facilitation**
Presynaptic neurons prepare vesicles containing neurotransmitters, priming them for release upon the arrival of an action potential. This process involves a complex molecular machinery that we will not discuss in detail here. However, it is important to note that this process is calcium-dependent. With each arriving presynaptic action potential, the level of presynaptic calcium increases.

If action potentials arrive in close succession, the calcium from one action potential adds to the residual calcium remaining from earlier ones. This cumulative increase in calcium enhances the probability of vesicle release, as long as vesicles remain available. Consequently, the postsynaptic excitatory postsynaptic potential (EPSP) becomes larger with successive, closely spaced action potentials.

**Depletion**
In contrast to facilitation, depletion refers to a decrease in synaptic efficacy due to the depletion of readily releasable vesicles. When a presynaptic action potential arrives, it triggers the release of neurotransmitters from vesicles, but the supply of these vesicles is limited. If action potentials occur at a high frequency, vesicles are released faster than they can be replenished. As a result, subsequent action potentials lead to fewer vesicles being available for release, reducing the postsynaptic excitatory postsynaptic potential (EPSP). Depletion is therefore a form of short-term plasticity that reflects a transient limitation in presynaptic resources. This effect depends on the balance between vesicle replenishment rates and the frequency of presynaptic activity.

## 1.1 Mathematical model

The following differential equations describe the model proposed by Tsodyks and Markramth for Short-term plasticity (STP).
The model uses two variables to describe the internal state of a neuron.
First, $x$ describes the fraction of available recources (vesicles in the presynapse). Upon the arrival of an action potential, $x$ decreases due to neurotransmitter depletion, while over time, it recovers to 1.0.

Secondly, $u$ describes the fraction of utilized resources (of the available resources). Upon the arrival of an action potential it increases due to calcium influx, while it decays back to zero over time. Formally:

$$
\frac{du}{dt} = -\frac{u}{\tau_f} + U(1 - u^-) \ \delta(t - t_{sp})
$$

$$
\frac{dx}{dt} = \frac{1 - x}{\tau_d} - u^+ x^- \ \delta(t - t_{sp})
$$

$$
\frac{dI}{dt} = -\frac{I}{\tau_s} + A u^+ x^- \ \delta(t - t_{sp})
$$

where $t_{sp}$ denotes the spike time and $U$ is the increment of $u$ produced by a spike. We define $u^-$ and $x^-$ as the values of $u$ and $x$ just _before_ the spike arrives and $u^+$ as the value of $u$ immediately _after_ the spike.

## 1.3 Assignments - Getting a mathematical intuition

You can try to understand the formulas intuitively before demonstrating the differential system that they describe using a simulation.

For example, the differential equation $\frac{du}{dt}$ describes how $u$ changes over time $t$.
For $u$, the change in time ($\frac{du}{dt}$) has two terms $-\frac{u}{\tau_f}$ and $U(1 - u^-) \delta(t - t_{sp})$.


Let's think about these terms:

**Assignment 1:** What does $-\frac{u}{\tau_f}$ describe?

**Assignment 2:** What does $U(1 - u^-) \ \delta(t - t_{sp})$ describe?

**Assignment 3:** Why use $u^-$ rather than $u$ in the second term?

**Assignment 4:** Given the solution from the last question, think about the two terms in $\frac{dx}{dt}$. Why use $u^+$ and $x^-$?

---

You should now have an intuition for the dynamics of the presynaptic neuron's internal states in the model proposed by Tsodyks and Markram
.
The last equation above describes how the current (EPSP amplitude) in the postsynaptic neuron is affected by STP dynamics.

$$
\frac{dI}{dt} = -\frac{I}{\tau_s} + A u^+ x^- \ \delta(t - t_{sp})
$$

**Assignment 5:** Think about the two terms in $\frac{dI}{dt}$. What do they describe?


<font color='red'>Solution Assignment 1</font>

The first term $-\frac{u}{\tau_f}$ describes the exponential decay of the signal $u$ over time.

This term makes $u$ decay to $0$ (without spikes).

<font color='red'>Solution Assignment 2</font>

The second term $U(1 - u^-) \delta(t - t_{sp})$ describes the influence of a spike.
It uses the Dirac delta function $\delta$ to put the entire mass at exactly $t_{sp}$, meaning the term is always $0$ except at exactly $t_{sp}$ it is $1$, putting the whole term to either $0$ or $U (1 - u^-)$.

This term describes what happens to $u$ in case of a spike.

<font color='red'>Solution Assignment 3</font>

At the time of a spike $t_{sp}$, $u$ is not defined because the spike happens by definition precisely at $t_{sp}$.
So at time $t_{sp}$, $u$ has _two_ values! A value before the spike $u^-$ and after the spike $u^+$.
This is because the spire is immediately at time $t_{sp}$, so there exists a before and after the spike at the time $t_{sp}$.
So we need $u^-$ (and $u^+$) to distinguish at time $t_{sp}$ about which $u$ we are talking about.
Due to the same reason, we also need a $x^-$ and $x^+$.

In the equation, $u$ caused the spike, so we need to take $u^-$, the value _before_ the spikem rather than $u^+$

We have discrete time steps if we simulate those formulas due to practical concerns.
So, in the code, we simulate the effects of the 1. term up to $t_{sp}$ to calculate $u^-$, then we calculate the effects of the 2. term at $t_{sp}$.


<font color='red'>Solution Assignment 4</font>

The first term increases $x$ using an exponential decay in $1-x$.
This term makes $x$ recover to $1$ (without spikes).

The second term, $x^- u^+$, describes what happens to $x$ in case of a spike.
More specifically, it represents the resources utilized through the spike.
Essentially, the fraction of the available resources _after_ the spike $x^+$ is the fraction of the available resources _before_ the spike $x^-$ ("what we had") times the fraction of resources utilized _after_ the spike $u^+$ ("what we use").


<font color='red'>Solution Assignment 5</font>

The first term $-\frac{I}{\tau_s}$ describes the exponential decay of the neuron current $I$ over time.

This term decays the output current to $0$ (without spikes). Similar to the first term in $\frac{du}{dt}.

The second term $A u^+ x^- \ \delta(t - t_{sp})$ describes what happens during a spike.
The output current jumps by $A$ times the utilized resources $u^+ x^-$. Conceptually, it is the negative of the second term in $\frac{dx}{dt}$.

## 1.3 Demo

Below is an interactive demonstration of the model for Short-term plasticity proposed by Tsodyks and Markramth.

You can play around with the sliders to change the parameters found in the equations above.

As discussed, two dynamics can be observed in STP:

1. Short-Term Facilitation (STF): Caused by calcium influxand accumulation, which in terms enhances the neurotransmitter release probability.
2. Short-Term Depression (STD): Results from neurotransmitter depletion due to action potentials arriving at too high freqyency.

Below, you have two buttons to set the parameters to fit one of these dynamics.

In [3]:
iplot_InoF_model()

### 1.4.1 Assignments
- **Assignment 6:** Try to understand the parameters for "SPD-dominated". What are the essential parameter changes? How can we increase the Short-term depression effect? Also, play around with the spike pattern to sharpen your intuition.
- **Assignment 7:**  Try to understand the parameters for "SPF-dominated". What are the essential parameter changes? How can we increase the Short-term facilitation effect?

<font color='red'>Solution Assignment 6</font>

For the "SPD-dominated" setting, we have the following parameters:

$x_0 = 1.0$,
$u_0 = 0.0$,
$\tau_s = 0.02$,
$\tau_f = 0.05$,
$\tau_d = 0.75$,
$U = 0.45$,
$A = 1.0$

For the "depression" effect, the most critical aspect is that $\tau_d$ is way bigger than $\tau_f$.
Therefore, the $u$ decays fast while the $x$ increases slowly.
To increase the "depression" effect, we can further widen this gap by making $\tau_d$ bigger or $\tau_f$ smaller, e.g., by setting $\tau_f=0.01$. Additionally, by increasing $U$, e.g., $U=1.0$, we increase the resources a spike utilizes.
This results in a massive neuron current, followed by depression (low neuron current).

Finally, if there is no spike for a while, the neuron recovers ($x$ increases), and a new spike at 0.99 triggers a significant neuron current again.

<font color='red'>Solution Assignment 7</font>

For the "SPF-dominated" setting, we have the following parameters:

$x_0 = 1.0$,
$u_0 = 0.0$,
$\tau_s = 0.02$,
$\tau_f = 0.75$,
$\tau_d = 0.05$,
$U = 0.15$,
$A = 1.0$

Compared to "SPD-dominated" the $U$ is smaller ($0.45 -> \0.15$), and $\tau_f$ and $\tau_f$ have values swapped.
The most critical aspect is that $\tau_f$ is way bigger than $\tau_d$.
To increase the "depression" effect, we can further widen this gap by making $\tau_f$ bigger or $\tau_d$ smaller, e.g., by setting $\tau_d=0.01$. Additionally, by decreasing $U$, e.g., $U=0.05$, we reduce the resources a spike utilizes.
    
Because we are in the SPF regime, the neuron recovers fast after each spike.

# 2. Spike-timing-dependent plasticity (STDP)

**Spike-timing-dependent plasticity (STDP)** is a neuronal mechanism that describes the <u>change in synaptic strength</u> based on the <u>temporal relationship between pre- and postsynaptic action potentials</u>, or "spikes."

If a <u>presynaptic spike consistently precedes a postsynaptic spike</u> within a short interval (10s of ms), the <u>synapse strengthens</u> (**potentiation**). Conversely, if the <u>presynaptic spike follows the postsynaptic spike</u>, the /u>synapse weakens</u> (**depression**). The result is, that connections are likely strenghtened if they have successfully contributed to the postsynaptic neuron's action potential.

## 2.1 Mathematical Model

The dynamics of weight changes observed in neurons showing **STDP** can be captured using mathematical models. Let $t_{pre}$ be the <u>time when the presynaptic spike</u> occurs and $t_{post}$ the <u>time of the postsynaptic spike</u>. We denote the temporal difference between these spikes as

$$
\Delta t = t_{post} - t_{pre} \text{, where } t > 0
$$

Since <u>STDP works</u> only if the pre- and postsynaptic neuron fired <u>action potentials within a narrow window of time</u> (if the time difference grows too big, the spikes of both neurons have likely no relationship to each other and should thus not inform changes in synaptic strength).

We know from experiments that the size of weight increase (or decrease) can be captured as a function of $\Delta t$. However, the **relationship is not linear** but **weights rather decreases exponentially** as the value of $\Delta t$ increases. Thus, the closer $t_{pre}$ and $t_{post}$ are together, the more the weights increase.

The positive change of a weight $w_{ij}$ (potentiation) connecting neuron $i$ (firing the presynaptic spike) to neuron $j$ (firing a postsynaptic spike) is defined as

$$
\Delta w_{ij} = A_+ \cdot \exp\left({\frac{-t}{\tau_+}}\right)\text{, where } \Delta w_{ij} \geq 0
$$

where $A_+$ denotes the maximal positive weight change and $\tau_+$ is a time constant.

However, if neuron $j$ fires a spike before neuron $i$, the weight decreases. In that case, $t_{pre}$ describes the firing time of neuron $j$, and $t_{post}$ denotes the firing time of neuron $i$. The negative weight change is defined as

$$
\Delta w_{ij} = A_- \cdot \exp\left({\frac{-t}{\tau_-}}\right) \text{, where } \Delta w_{ij} \leq 0
$$

where $A_-$ denotes the maximal negative weight change and $\tau_-$ is a time constant.


## 2.2 Simulation

In the section below we investigate the mathematical model of STDP. You will find two tabs:

- In the left tab, you can change the configuration and adjust the parameters $A_+$, $A_-$, $\tau_+$, and $\tau_-$. Note that after changing a configuration, you need to save it by pressing the `Update Config` button. However, this will overwrite any previously captured data.
- In the right tab, you can define the timing of the spike of neuron $j$ (relative to the neuron $i$) and submit it. After submission, the plots below will be updated.

After submitting a spike, three outputs below will be updated. They show:

**Calculation**

In the first row, you see the calculation of the weight change $\Delta w_{ij}$. This calculation is updated based on your configuration and the timing of the pre- and postsynaptic spike timing.

**Spike Timing**

In the second row, you can observe the timing of the spikes. The red line depicts the spike of neuron $i$ and the blue line the spike of neuron $j$. If the spike of neuron $i$ is before the spike of neuron $j$ (red before blue), $\Delta w_{ij}$ increases, otherwise, it decreases.

**Weight Change**

In the third row, we keep track of the weight changes. On the y-axis, you can observe the weight change, while on the x-axis, you can see the time difference $\Delta t$. If there are multiple registered spikes, we apply a linear interpolation between them.

---

You can add (submit) as many pre-post spike relationships as you want - each time you do so, the plots will be updated. Keep in mind, that once you change the parameters in the first tab, you have to start over.

In [ ]:
iplot_stdp_model()

#### 2.2.1 Assignments: Calculation by Hand

**Assignment 8**: Select a random (negative) time for neuron $i$ so that it fires a spike before neuron $j$. Without pressing the submit button, calculate the weight change $\Delta w_{ij}$. After calculating, press submit and compare your result with the solution shown in the plot. Is your calculation identical to the solution shown by the algorithm? Repeat this exercise, but this time, make neuron $j$ fire before neuron $i$.

**Assignment 9**: Find the submitted value in the plot in the third row.

---

#### 2.2.2 Assignments: Create several spikes

**Assignment 10**: Define and submit $10$ different $\Delta t$ where neuron $i$ fires **before** neuron $j$ and $10$ cases where neuron $i$ fires **after** neuron $j$. How does the time-weight change curve look like?

**Assignment 11**: Repeat the experiment after adjusting the parameters in the first tab. Answer the questions below:

- How does $A_+$ and $A_-$ influence the weight change?
- What is the role of $\tau_+$ and $\tau_-$?

<font color='red'>Solution Assignment 11</font>

$A_+$ and $A_-$ determine the maximal weight change. If you increase $A_+$, the weight change will be more significant for positive $\Delta t$ values. If you increase $A_-$, the weight change will be more significant for negative $\Delta t$ values.

$\tau_+$ and $\tau_-$ determine the time constant of the weight change. If you increase $\tau_+$ or $\tau_-$,, the weight change will be more spread out over time.

# 3. Encoding Odor Identity

In this third exercise, we will simulate a neural network inspired by the structure of the anterior piriform cortex (APC), a brain region responsible for encoding odor identity in mammals.<br>
This exercise is divided into two sections: The first section introduces the biological basis of odor identity encoding, while the second section simulates it in a neural network. For each section, you will be asked to answer the questions at the end (one or two sentences are enough).

#### Acronyms
- **APC**: Anterior Piriform Cortex
- **OE**: Olfactory Epithelium
- **OSN**: Olfactory Sensory Neuron
- **OR**: Olfactory Receptor
- **OB**: Olfactory Bulb
- **LOT**: Lateral Olfactory Tract
- **MTC**: Mitral/Tufted Cell
- **PN**: Pyramidal Neuron
- **PV-IN**: Parvalbumin Interneuron
- **SST-IN**: Somatostatin Interneuron
- **VIP-IN**: Vasoactive Intestinal Peptide Interneuron
- **LTP**: Long Term Potentiation

## 3.1 Biology

### 3.1.1 Smell
The smell of freshly brewed coffee is the result of various odorants activating olfactory sensory neurons (OSNs) by binding to olfactory receptors (ORs) in the nasal olfactory epithelium (OE) (Figure 1 $\textbf{A}$). Each OSN expresses one specific OR-subtype which is tuned to particular physicochemical properties of molecules.  
Axons of OSNs expressing identical OR-subtypes converge onto the same glomerulus in the olfactory bulb (OB), creating a highly ordered input structure.


<figure>
    <center>
        <img src="https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/Figures/IFig1.png"/>
        <figcaption align = "justify"> <b>Figure 1: Mouse olfactory system and odor identity encoding in the APC: A) </b>  Odorants bind to ORs on OSNs. Axons of OSNs expressing identical OR subtypes converge onto the same glomerulus. In the OB, signal preprocessing occurs before MTCs relay the input to PNs in the APC. Inhibitory neurons are shown in red. Adapted from <a href="#ref">Shepherd (1991)</a> with inspiration from <a href="#ref">Uchida et al. (2014)</a>. <b> B1) </b> Afferent input via L1a (axons of MTCs) activates a subset of PNs (triangles), while many neurons either remain below their threshold level or are not activated at all. <b> B2) </b> Active PNs recruit additional cells into the neuronal ensemble through recurrent intracortical connections, but only if these neurons have already received input via afferents. Recurrent input alone is not strong enough to recruit inactive neurons. </figcaption>    
    </center>
</figure>



### 3.1.2 Anterior piriform Cortex \& odor identity encoding
The piriform cortex, a three-layered paleocortex, receives direct input from the OB via axons of Mitral and Tuftet cells (MTCs). Interestingly, the topographical order of the input, meticulously maintained throughout the OB, is lost as the axons of the MTCs project diffusely to the APC.

Pyramidal neurons (PNs) in the APC receive afferent input from MTCs at the distal part of their apical dendrite (Figure 1 $\textbf{B1}$). To consistently activate a PN, it takes 5-10 co-active MTCs.
As a result, only a tiny subset of PNs fire, while the remaining cells may experience depolarization below their activation threshold. The active subset of neurons recruits additional PNs into the ensemble through recurrent intracortical connections, targeting the PNs dendrites closer to the soma (Figure 1 $\textbf{B2}$). The recurrent activation is strong enough to activate the previously depolarized neurons but too weak to recruit PNs that did not receive any signal from the OB at all. As network activity intensifies, recurrent inhibition is triggered, suppressing the response of neurons recruited later in the process. Consequently, odor identity is encoded by unique ensembles of active neurons in a concentration-invariant manner within the APC.

The strengthening of recurrent intracortical synapses between PNs is hypothesized to be the driving force behind the formation of long-lasting ensemble odor representations.

### 3.1.3 Disinhibitory circuit

With their synapses close to the soma of PNs, parvalbumin interneurons (PV-INs) exert rapid and potent inhibition and are thought to be critical in establishing the sparse and distributed activity patterns within PN ensembles.
The inhibitory synapses of somatostatin-positive interneurons (SST-INs) on the proximal apical dendrite of PNs, however, are optimally located to regulate intracortical synaptic plasticity. [Canto-Bustos et al. (2022)](#ref) showed that long-term potentiation (LTP) in those PN-to-PN synapses can be gated by inactivating SST-INs and that vasoactive intestinal polypeptide (VIP) expressing cells are responsible for strong inhibition of those SST-INs.
A teaching signal that activates this disinhibitory VIP-to-SST-to-PN circuit could shape neuronal ensemble formation and may be recruited on novel, salient, or emotionally relevant odors.

<figure>
    <center>
        <img src="https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/Figures/IFig3.png"/>
        <figcaption align = "center"> <b>Figure 2: Simplified APC microcircuit: </b> Odor input arrives diffusely at the apical dendrites of PNs. PNs recurrently project to other PNs uniformly throughout the APC. Connections to inhibitory SST- and PV-INs, lead to feedback inhibition. A teaching signal can activate the disinhibitory VIP-SST-PN circuit and open a window for LTP in intracortical PN-to-PN connections. </figcaption>    
    </center>
</figure>


Based on this model, we will develop a simplified version of the APC microcircuit (Figure 2) and explore the effect of a teaching signal that activates the disinhibitory VIP-to-SST circuit during odor learning.


## 3.2 Assignments


**Assignments 12:** Can different OR subtypes show affinity to the same molecule?

**Assignments 13:** Do you expect that without a teaching signal, odor identity is not learned at all? Name a learning algorithm that does not require a teaching signal.

<font color='red'>Solution Assignment 12</font>

Yes, since the receptors are tuned to certain features of molecules, not the whole molecule itself.

<font color='red'>Solution Assignment 13</font>

Odor identity learning is still possible, but slower. Hebbian learning - Anti-Hebbian learning

## 3.3. Simulation

### 3.3.1 Network
<a id='eq'></a>
We will use a rate-based model, which means that we will not model the spiking of the neurons themselves. Rather, we will model the membrane potential of the neurons using first-order differential equations and then use a non-linear activation function to convert it into instantaneous activity. Our network will consist of 50 Pyramidal neurons (PN's), 8 SST neurons, 4 PV neurons and 1 VIP neuron.

You do not need to understand the model. However, interested students are encouraged to look at the differential equations and the code of the class `APC_circuit` inside the file `utils_ex10/odor.py`

<details>
<summary><b>Differential Equations</b></summary>

The following differential equations describe the membrane potential of our neurons:


\begin{align}
\frac{d\vec{V}_{PN}}{dt} ={} & W_{Input}^{PN} \cdot \vec{x}(t) + W_{PN}^{PN} \cdot \vec{r}_{PN}(t) -k_{PN} \vec{V}_{PN}(t)\\ & - \min\left(W_{Input}^{PN} \cdot \vec{x}(t),W_{SST}^{PN} \cdot \vec{r}_{SST}(t)\right) - W_{PV}^{PN} \cdot \vec{r}_{PV}(t) \\[1em]
\frac{d\vec{V}_{PV}}{dt} ={} & W_{PN}^{PV} \cdot \vec{r}_{PN}(t)-k_{PV} \vec{V}_{PV}(t)\\[1em]
\frac{d\vec{V}_{SST}}{dt} ={} & W_{PN}^{SST} \cdot \vec{r}_{PN}(t) -k_{SST} \vec{V}_{SST}(t) - W_{VIP}^{SST} \cdot \vec{r}_{VIP}(t) \\[1em]
\frac{d\vec{V}_{VIP}}{dt} ={} & \vec{\tau}(t) -k_{VIP} \vec{V}_{VIP}(t) \\[1em]
\vec{r}_X(t) ={} & ReLU(\vec{V_X}(t)) = \max(0,\min((\delta_X \cdot \vec{V_X}(t) - \theta_X), r_X^{max}))
\end{align}


<br>
<br>


| Parameter     | Value         | Name in Code  | Explanation   |
| :------------ | :------------ | :------------ | :------------ |
|  $\vec{V}_{X}$ |  $\mathbb{R}$ | V\_{X} | Membrane potential of X |
|  $r_{X}(t)$ | $\mathbb{R}_+^{n}$ | Activity\_X | Activity (rate) of X |
|  $r_X^{max}$ | $\mathbb{R}_+$ | Rmax\_X | Maximal activity (rate) of X |
|  $k_{X}$ | const.  | k\_X | EPSP decay rate for X|
|  $W_{X}^{Y}$ | $\mathbb{R}_+^{n \times m}$ | weightX\_Y |  Weight matrix for $X\to Y$ connection <br> with $n = \#Y$ and $m = \#X $ <br> only $W_{PN}^{PN} $ is time dependent |
|  $\vec{x}(t)$ | $\mathbb{N}_+^n$ | input | Input at time $t$ |
|  $\tau(t)$ |  const. | teacher | Magnitude of teacher input |
|  $\bar{r}(t)$ | $\mathbb{R}_+$ |  r\_avrg | Average rate from $T-t$ to $t$ |
|  T | 100 $ms$ | timeAvrg | Time-window of integration  <be> for weight update |
|  $\alpha$ | 0.8 | alpha | Rate of regularization in OJA |
|  $\lambda$ | $1.2 \times 10^{-6}$ | learning\_rate | Learning rate |
|  $\theta$ | $\mathbb{R}_+$ | Thresh\_X | Activity threshold |
|  $\delta$ | $\mathbb{R}_+$ | Gain\_X | Activity gain |
|  $dt$ | 1 ms | -- | Time increment |

$$X,Y \in \{PN,SST,PV,VIP\}$$



We model the differential equations using the [forward Euler](https://en.wikipedia.org/wiki/Euler_method) method with a time-step of $dt = 1~ms$.

</details>

### 3.3.2 Odor
We model odor as binary vectors. A "$1$" indicates the presence of a certain odorant in the odor, and a "$0$" indicates its absence. For simplicity, we restrict the size of an odor to $10$ different odorants and define that an odor always consists of $2$ odorants.
One of the odors will be linked to a teaching signal and one odor will function as a control. These two odors will be presented in an alternating fashion to our neuronal network. Further, random odors are presented to the network to test the noise resistance of the network (Figure 3).


<figure>
    <center>
        <img src="https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex10/Figures/MFig2.png"/>
        <figcaption align = "justify"> <b>Figure 3: Odor distribution.</b> Blue odors are linked to a teaching signal and darkgray inputs serve as controls. Additional random odors are presented during training to evaluate learning with noisy input.
 </figcaption>    
    </center>
</figure>

## 3.4 Simulation

The function below implements the network with Odor input. We will not go into the details of the model itself but observe what happens when such a network is trained. We have three different modes.

**No Learning**:

The network is initialized randomly and the Odor signal is fed through the network. Thereby, the weights remain the same

**Hebbian Learning**:

One of the simplest unsupervised learning rules is Hebbian learning. It is defined as:

$$\frac{\Delta w}{\Delta t} = \lambda \cdot x \cdot y $$
$$ w(t)  = w(t-1) +  \Delta w(t) $$

Where $x = r(t-1)$ is the presynaptic activation, $y = r(t)$ is the postsynaptic activation, $w$ is the weight between the neurons and $\lambda$ is the learning rate.

We implement learning only for the recurrent PN-to-PN connections.

**Oja Learning**:

As you know, neurons have limited resources and the weights between synapses (e.g., the number of AMPA receptors) cannot increase indefinitely. Furthermore, neurons tend to normalize their activity to a given baseline. A Hebbian-based learning rule that incorporates this concept is Oja's learning rule:

<a id='oja'></a>
$$\Delta W(t) = \frac{\lambda}{T}\left[\int_{t - T}^{t} \vec{r}(u) \vec{r}^\top(u-1) du - W \alpha \int_{t - T}^{t} \overline{r}(u)^2 du\right]$$

Weight update:
$$ W(t)  = W(t-1) +  \Delta W(t) $$

To make the learning rule smoother, we average over a time window specified in the variable `self.timeAvrg` which is set to $100~ms$. Since we are modeling time in discrete steps, we are actually summing and not integrating.

### 3.4.1 Simulation

Execute the cell below. Read the eplanatory text and attend to **assignments 14-18** embedded within the tabs of the visualization below.

In [4]:
odor_model()

Output()

<font color='red'>Solution Assignment 14</font>

Without training, the activity patterns are identical at the beginning and at the end of the simulation, as they only depend on the input pattern. With Hebbian learning, the activity patterns change over time, as the network learns to represent the input pattern. However, at the very end, all neurons are either inactive or highly active - Please look at the next tabs to find out why this happens. With Oja's learning rule, the activity patterns are more stable (see next tabs)

<font color='red'>Solution Assignment 15</font>

Without training, the activity does not change and is therefore a flat line. With Hebbian learning, it looks like all inhibitory neurons and all PN's that were recruited into an odor encoding neuronal ensemble fire at their maximum firing rate. With Oja's learning rule, the activity looks much more realistic.

<font color='red'>Solution Assignment 16</font>

With Hebbian learning, some of the weights, and therefore some of the activities (see previous tabs), explode and achieve weights around 700. With Oja's learning rule, the weights are regularized and stay in a regime that does not lead to such strong activations.

<font color='red'>Solution Assignment 17</font>

Odor intensity is encoded in the dynamics of the neuronal ensembles encoding odor identity.

<font color='red'>Solution Assignment 18</font>

Here is the translated sentence adjusted for use in a student assignment solution: --- Since we are plotting the mean of the weights, the L2 regularization (only applicable for Oja's rule) causes the weights to grow faster for smaller ensembles (ensembles for the control odor often consist of fewer neurons).
